In [2]:
import { env } from './env.ts'

In [3]:
import { TextLoader } from "langchain/document_loaders/fs/text";
const loader = new TextLoader('../dataset/qiu.txt');
const docs = await loader.load()

In [ ]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 500,
  chunkOverlap: 100,
});

const splitDocs = await splitter.splitDocuments(docs);

In [5]:
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  configuration: {
    baseURL: env["QWEN_BASE_URL"],
    apiKey: env["QWEN_API_KEY"],
  },
  model: env["QWEN_EMBEDDINGS_MODEL"],
});

In [6]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorStore = new MemoryVectorStore(embeddings);
for (let i = 0; i < splitDocs.length; i += 10) {
  await vectorStore.addDocuments(splitDocs.slice(i, i + 10));
}

In [ ]:
const retriever = vectorStore.asRetriever(2);
const res = await retriever.invoke("原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论")